<a href="https://colab.research.google.com/github/Luensmann/Bachelorarbeit/blob/main/Pre%20Processing/Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset
import json

from huggingface_hub import notebook_login
notebook_login()

In [ ]:
def load_data(file):
  train = file + "-train-ibo.json"
  test = file + "-test-ibo.json"
  data_files = {"train": train, "test": test}
  raw_data = load_dataset("json", data_files=data_files)

  raw_datasets = raw_data["train"].train_test_split(train_size=0.8, seed=42)
  raw_datasets["validation"] = raw_datasets.pop("test")
  raw_datasets["test"] = raw_data["test"]
  return(raw_datasets)

In [ ]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if(type(label) != type(2)):
              print(label, type(label), type(2))
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], add_special_tokens=False, is_split_into_words=True, truncation=True, max_length=512
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, trunction=True , model_max_len=512)

In [ ]:
files =['amia', 'Variome','tmvar', 'SETH', ]

In [ ]:
for file in files:
    # load dataset
    filename_in = "Brizape/" + file + "_split_0404"
    raw_data = load_dataset(filename_in)
    print(raw_data)
    
    # plit train to train, dev
    raw_datasets = raw_data["train"].train_test_split(train_size=0.8, seed=42)
    raw_datasets["validation"] = raw_datasets.pop("test")
    raw_datasets["test"] = raw_data["test"]
    print(raw_datasets)
    raw_datasets.push_to_hub("Brizape/" + file + "_split_0404_dev")
    
    tokenized_datasets = raw_datasets.map(
      tokenize_and_align_labels,
      batched=True,
          #remove_columns=raw_datasets["train"].column_names,
      )
    print(tokenized_datasets)

    tokenized_datasets.push_to_hub("Brizape/" + file + "_tokenized_split_0404_dev")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/420 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1142 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to C:/Users/pc/.cache/huggingface/datasets/Brizape___parquet/Brizape--amia_split_0404-dc44dbf5fa9136eb/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

...

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Pushing split test to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]